In [1]:
import numpy as np
import pandas as pd  

data_file_csv1 = 'inner_20_0_new.csv'
data_file_csv2 = 'ball_20_0_new.csv'
data_file_csv3 = 'outer_20_0_new.csv'
data_file_csv4 = 'comb_20_0_new.csv'
data_file_csv5 = 'health_20_0_new.csv'
origin_data1 = pd.read_csv(data_file_csv1)
origin_data2 = pd.read_csv(data_file_csv2)
origin_data3 = pd.read_csv(data_file_csv3)
origin_data4 = pd.read_csv(data_file_csv4)
origin_data5 = pd.read_csv(data_file_csv5)

print(origin_data1.shape)
print(origin_data2.shape)
print(origin_data3.shape)
print(origin_data4.shape)
print(origin_data5.shape)
origin_data1.head()

(1048560, 8)
(1048560, 8)
(1048560, 8)
(1048560, 8)
(1048560, 8)


,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
0,-0.117462,-0.001712,0.001460,0.007768,0.002619,0.003544,0.003097,-0.007119
1,-0.122317,-0.000914,-0.004050,-0.006717,0.003122,-0.012211,0.005323,-0.002731
2,-0.121311,0.004437,0.003101,0.007753,0.006050,0.003830,-0.007553,0.008774
3,-0.120751,-0.007136,-0.002774,0.002573,0.006198,0.001988,-0.001854,0.002958
4,-0.117236,-0.000008,0.000765,-0.007696,0.002301,0.004705,0.000281,0.005544


In [2]:

data_column = 'Column3'
columns_name = ['inner','ball','outer','comb','health']
file_names = ['inner_20_0_new.csv', 'ball_20_0_new.csv', 'outer_20_0_new.csv', 'comb_20_0_new.csv', 'health_20_0_new.csv']
combine_data = pd.DataFrame()
for index in range(5):
    # read CSV file
    data = pd.read_csv(file_names[index])
    dataList = data[data_column]
    combine_data[columns_name[index]] = dataList
print(combine_data.shape)
combine_data

(1048560, 5)


,inner,ball,outer,comb,health
0,0.001460,-0.007886,-0.000808,-0.392166,-0.458107
1,-0.004050,0.003791,0.002338,-0.379996,-0.449544
2,0.003101,-0.002126,-0.000834,-0.389422,-0.454772
3,-0.002774,-0.001577,0.001218,-0.388949,-0.458454
4,0.000765,0.002419,0.000473,-0.386126,-0.450946
...,...,...,...,...,...
1048555,-0.001255,-0.005304,0.003202,0.003767,-0.002427
1048556,-0.003281,0.002497,-0.001090,0.000229,0.000675
1048557,0.003114,-0.001674,0.003688,0.000527,0.004184
1048558,-0.007981,0.008260,-0.004952,0.000223,0.000635


In [3]:
combine_data.set_index('inner',inplace=True)
combine_data.to_csv('combine_data.csv')
combine_data.shape

(1048560, 4)

In [4]:
import numpy as np
import pandas as pd
import sklearn
from joblib import dump, load

# step 1024 
# window = 1024  step = 512  

def split_data_with_overlap(data, time_steps, lable, overlap_ratio=0.5):

    stride = int(time_steps * (1 - overlap_ratio))  
    samples = (len(data) - time_steps) // stride + 1  
    Clasiffy_dataFrame = pd.DataFrame(columns=[x for x in range(time_steps + 1)])  
    data_list = []
    for i in range(samples):
        start_idx = i * stride
        end_idx = start_idx + time_steps
        temp_data = data[start_idx:end_idx].tolist()
        temp_data.append(lable)  
        data_list.append(temp_data)
    Clasiffy_dataFrame = pd.DataFrame(data_list, columns=Clasiffy_dataFrame.columns)
    return Clasiffy_dataFrame

def normalize(data):

    s= (data-min(data)) / (max(data)-min(data))
    return  s

def make_datasets(data_file_csv, split_rate = [0.7,0.2,0.1]):

    origin_data = pd.read_csv(data_file_csv)
    time_steps = 1024 
    overlap_ratio = 0.5  

    samples_data = pd.DataFrame(columns=[x for x in range(time_steps + 1)])  

    label = 0

    for column_name, column_data in origin_data.items():

        split_data = split_data_with_overlap(column_data, time_steps, label, overlap_ratio)
        label += 1 
        samples_data = pd.concat([samples_data, split_data])

    samples_data = samples_data.sample(frac=1).reset_index(drop=True)
    sample_len = len(samples_data) 
    train_len = int(sample_len*split_rate[0]) 
    val_len = int(sample_len*split_rate[1]) 
    train_set = samples_data.iloc[0:train_len,:]   
    val_set = samples_data.iloc[train_len:train_len+val_len,:]   
    test_set = samples_data.iloc[train_len+val_len:,:]   
    return  train_set, val_set, test_set, samples_data

train_set, val_set, test_set, samples_data = make_datasets('combine_data.csv')


dump(train_set, 'train_set') 
dump(val_set, 'val_set') 
dump(test_set, 'test_set') 

['test_set']

In [5]:
print(type(samples_data))
samples_data.set_index(0, inplace=True)
samples_data.to_csv('samples_data.csv')
samples_data.shape

<class 'pandas.core.frame.DataFrame'>


(10230, 1024)

In [6]:
samples_data.shape

(10230, 1024)

In [7]:

import torch

def make_data_labels(dataframe):

    x_data = dataframe.iloc[:,0:-1]
    y_label = dataframe.iloc[:,-1]
    x_data = torch.tensor(x_data.values).float()
    y_label = torch.tensor(y_label.values.astype('int64')) 
    return x_data, y_label


train_set = load('train_set') 
val_set = load('val_set') 
test_set = load('test_set') 


train_xdata, train_ylabel = make_data_labels(train_set)
val_xdata, val_ylabel = make_data_labels(val_set)
test_xdata, test_ylabel = make_data_labels(test_set)

dump(train_xdata, 'train_xdata')
dump(val_xdata, 'val_xdata')
dump(test_xdata, 'test_xdata')
dump(train_ylabel, 'train_ylabel')
dump(val_ylabel, 'val_ylabel')
dump(test_ylabel, 'test_ylabel')

['test_ylabel']

In [8]:
print('data shape:')
print(train_xdata.size(), train_ylabel.size())
print(val_xdata.size(), val_ylabel.size())
print(test_xdata.size(), test_ylabel.size())

data shape:
torch.Size([7161, 1024]) torch.Size([7161])
torch.Size([2046, 1024]) torch.Size([2046])
torch.Size([1023, 1024]) torch.Size([1023])
